# modeling16-FiD-encoder-sentence-level-classifier-BuildSelectionDataSet
 - Making selection dataset from incremental results

In [1]:
import numpy as np
import pathlib
from pprint import pprint
import pandas as pd
from util import utils
import re
import os

## Get dataset for incremental Test
- README.md method 2,3
    - Method1 describes previous ranking system

## Definitions of Positive, Relevant, Damaging, Irrelevant  with respect to retriever (check out Meeting#3.ipynb)
    - Previous Definition of relevant passages are vague.
    - What is the definition of relevance?
        - Passages are retrived from the step?
    - Here we are going to dicuss the definition of each paradigm

### Previous passages are divded into two categories with respect to query
### Assumption 
    - higher the similarity score higher the the performance on downstream tasks
        - similarity score has positive correlation with positive passage
    - low similarity socre considered to be irrelevant degrades the output
    - Therefore, retrieval is a process of extracting positive passage from the relevant set
    - by shrinking the search space by top-k     
####  1. Positive passages 
        : passages that include correct answer
        : sometimes multiple passages are required
#### 2. Relevant passages (Relevant set : R)
        : passages have high possibilities in containing corret answer with repect to query/claim
        : usually measured by similariy score
#### 3. Irrelevant passages (Irrelevant set : I) 
        : passages have low probabilities in containing corret answer with repect to query/claim
        : usually measured by similariy score
    


### New Definitions are now divided into three categories with respect to query

### Assumption
    - Previous works focus on the re-ranking of retrieved passages, which based on the assumption that taking top-k passages with higher similarity scores will show higher performance in the downstream tasks.
    - ; however, we have been shown that among those retrieved passages with higher similarity scores actually downgrades the final output which could have been produce the correct answer without them.
    - So those passages are what we called damaging passges that have to be removed not just re-ranked by its score.

#### 1. Positive passages 
        : passages that produce correct inference on downstream task
        : sometimes multiple passages are required
        
#### 2. Relevant passages = R1
    : passages retrieved from query that does not present adversarial effect when previous inference is correct
    : passages with high possibility inlucde corret answer with repect to query/claim
    : usually measured by similariy score
    : Subset of of R
    
#### 3. Damaging passages = D1
    : passages retrieved from query that produce wrong output when preivous inference is correct
    : passages with high possibility inlucde corret answer with repect to query/claim
    : HOWEVER it degrades the inference output
    : usually measured by similariy score
    : Subset of of R
    
#### 4. Irrelevant passages
    : passages retrieved from query that does not change the answer when previous inference is not correct
    : when there is preivous inference result and current output is not correct, we call it irrelevant
    - Passages retrieved by retriever that have no impacts for inference
    - usually measured by similariy score

## Incremental Testing Setting

### 3 Method (Check out README.md)

### Method 2 (2nd method of in README.md)
    - The sooner, The better approach
      - Keep the Positive Context in order
    - Since we testing it, let's keep sample size of 5
    - When there is no Exact Mathcing during the incremental inference e.g.) em_pattern = '00000'
        - Keep the whole context so that those cases will have False on Exact Match values
        
    - Patterns: 
        - first 1 : positive
        - first 0 : irrelevant 
            - we know that first is the positive context that includes answer
            - with that perspective, it is relevant in theory
            - ; however, dataset is created via BLEU score on two different corpus.
            - If FiD does not correctly infer the output, 
            - context what we concieved of artificial positive context is actually irrelevant to the query.
            - Also when realistic scenario, we don't know whether the first context contains the answer
        - 01 pattern : Positive 
        - 11 pattern : Strict Positive(relevant) or Naive Positive(positive)
        - 00 pattern : Strict Damaging(irrelevant) or Naive Damaging(damaging) 
        - 10 pattern : Damaging
        
        - 11 pattern : relevant vs positive
            Strict Positive
                Under strict rule : consider it as relevant ?(or irrelevant?)
                                in terms of strictly limiting the number of positive passages
            Naive Positive
                Under naive rule : consider it as positive
                                in that this would increase the number of positive passages
                
        - A00 pattern : irrelevant vs damaging
            if '1' does not occured in A, currnet passage is irrelevant
            if '1' occurred in A, current passage is damaging either irrelevant 
                Strict Damaging
                    Under strict rule : consider it as irrelevnat 
                                    in terms of strictly limiting the number of damaging passage
                Naive Damaging
                    Under naive rule : consider it as damaging 
                                    in that this would increase the number of damaging passages
        
    - Options1 : Removes only damaging
    - Options2 : Removes damaging + irrelevant
    - Options3 : Removes damaging + relevant
    - Options4 : Removes damaging + irrelevant + relevnat
        

### Total Trials 16
    1st Trial - Options1(Remove only Damaging) + Strict Positive + Strict Damaging
    2nd Trial - Options1(Remove only Damaging) + Strict Positive + Naive Damaging
    3rd Trial - Options1(Remove only Damaging) + Naive Positive + Strict Damaging
    4th Trial - Options1(Remove only Damaging) + Naive Positive + Naive Damaging
    
    5th Trial - Options2(Remove damaging + irrelevant) + Strict Positive + Strict Damaging
    6th Trial - Options2(Remove damaging + irrelevant) + Strict Positive + Naive Damaging
    7th Trial - Options2(Remove damaging + irrelevant) + Naive Positive + Strict Damaging
    8th Trial - Options2(Remove damaging + irrelevant) + Naive Positive + Naive Damaging
    
    9th Trial - Options3(Removes damaging + relevant) + Strict Positive + Strict Damaging
    10th Trial - Options3(Removes damaging + relevant) + Strict Positive + Naive Damaging
    11th Trial - Options3(Removes damaging + relevant) + Naive Positive + Strict Damaging
    12th Trial - Options3(Removes damaging + relevant) + Naive Positive + Naive Damaging
    
    13th Trial - Options4(Removes damaging + irrelevant + relevant) + Strict Positive + Strict Damaging
    14th Trial - Options4(Removes damaging + irrelevant + relevant) + Strict Positive + Naive Damaging
    15th Trial - Options4(Removes damaging + irrelevant + relevant) + Naive Positive + Strict Damaging
    16th Trial - Options4(Removes damaging + irrelevant + relevant) + Naive Positive + Naive Damaging
        
        
    - Since Options4 removes  "damaging + irrelevant + relevant"
        - there will be no difference between Strict Damaging and Naive Damaging

    13, 14th Trial -> Options4(Removes damaging + irrelevant + relevant) + Strict Positive
    15, 16th Trial -> Options4(Removes damaging + irrelevant + relevant) + Naive Positive
    
    - Since Options2 removes  "damaging + irrelevant"
        - there will be no difference in terms of the input between Strict Damaging and Naive Damaging
    5, 6th Trial -> Options2(Remove damaging + irrelevant) + Strict Positive -> new 5th
    7, 8th Trial -> Options2(Remove damaging + irrelevant) + Naive Positive -> new 6th
    - Since Options2 include both "positive + relevant"
        - There will be no difference in terms of the input between Strict Positive and Naive Positive
        - I found out during checking
    new 5th, new 6th -> Options2(Remove damaging + irrelevant) 
    
    - Options4(Removes damaging + irrelevant + relevant) + Strict Positive
        - means one positive only thus same as "pos1_ctx1" -> nope there might be non-consecutives
        - e.g.) 01011 -> 2 strict positive cases
        - BUT "pos1_ctx1" = baseline could be our baselines
    Options4(Removes damaging + irrelevant + relevant) + Strict Positive -> equal to "pos1_ctx1" 
    

    
### Final Total Trials 11
    1st Trial - Options1(Remove only Damaging) + Strict Positive + Strict Damaging
    2nd Trial - Options1(Remove only Damaging) + Strict Positive + Naive Damaging
    3rd Trial - Options1(Remove only Damaging) + Naive Positive + Strict Damaging
    4th Trial - Options1(Remove only Damaging) + Naive Positive + Naive Damaging
    
    5th Trial - Options2(Remove damaging + irrelevant) + Strict Positive + Strict Damaging 
        - The one also represents other three cases
            - Options2(Remove damaging + irrelevant) + Strict Positive + Naive Damaging
            - Options2(Remove damaging + irrelevant) + Naive Positive + Strict Damaging
            - Options2(Remove damaging + irrelevant) + Naive Positive + Naive Damaging
    
    6th Trial - Options3(Removes damaging + relevant) + Strict Positive + Strict Damaging
    7th Trial - Options3(Removes damaging + relevant) + Strict Positive + Naive Damaging
    8th Trial - Options3(Removes damaging + relevant) + Naive Positive + Strict Damaging
    9th Trial - Options3(Removes damaging + relevant) + Naive Positive + Naive Damaging
    
    10th Trial - Options4(Removes damaging + irrelevant + relevant) + Strict Positive 
        - The one represents cases 
            - Options4(Removes damaging + irrelevant + relevant) + Strict Positive + Strict Damaging
            - Options4(Removes damaging + irrelevant + relevant) + Strict Positive + Naive Damaging
        - almost equal to "pos1_ctx1" e.g.) 01011 -> 2 strict positive cases
        - "pos1_ctx1" = baseline could be our baselines
        
    11th Trial - Options4(Removes damaging + irrelevant + relevant) + Naive Positive    
        - The one represents cases 
            - Options4(Removes damaging + irrelevant + relevant) + Naive Positive + Strict Damaging
            - Options4(Removes damaging + irrelevant + relevant) + Naive Positive + Naive Damaging
    
    
### Estimation 
    - Since retrieved passages have high similarity score, retrieved passages are higly likely to form damaging 
    passages with repect to query. 
    - Minimizing the risk of exposing query to damaging passages would be favorabe to FiD
    - The simplest way to reduce to the probability of such occurences is limiting the number of input passages

    - I think below strategy will hold the best result 
        1) Options2(Remove damaging + irrelevant)
            * As we saw from result from FiD result with Random Sampling, 
            * FiD is powerful distinguish the positive from irrelevants.
            * Even though we loose stronger boundary that might be helpful to inference, 
            * It will be (frivolous/trivial/not consequential)
        2) Strict Positive (relevant) 
            * Strict Positive puts a rigorous boundary for passages, meaning less positive passages
        3) Naive Damaging(damaging) 
            * Unlike Strict Positive, we can set up a lenient standard for damaging to lessen the size of input. 
            * Increasing Damaging passages by definition results in decreasing number of inputs

### Method2
    - Same approach but in reciprocal order
      - We know that FiD is order invariant but this is for checking 
      - don't need to test on whole trials

# Input
    - result from 5-1
    - /data/philhoon-relevance/FiD/results/KILT_BM25_NQ/incremental_result/pos1_ctx5.json

# Trials
    - Need to check FiD input when there are less ctxs than n_context

In [2]:
# sample_size = 100
# input_file = f'/data/philhoon-relevance/FiD/results/NQ_CONTRIEVER/DEV/incremental_result_100/ctx{sample_size}.json'

In [3]:
# input_file = utils.open_json('/data/philhoon-relevance/FiD/results/KILT_BM25_NQ/incremental_result/pos1_ctx5.json')

### format
```python
{
    'id' : str()
    'answers' : list()
    'ctxs' : list(dict)
    'questions' : str()
}
```

#### Test Code 

### Checking on Final Total Trials 12

    1st Trial - Options1(Remove only Damaging) + Strict Positive + Strict Damaging (checked)
    2nd Trial - Options1(Remove only Damaging) + Strict Positive + Naive Damaging (checked)
    3rd Trial - Options1(Remove only Damaging) + Naive Positive + Strict Damaging (checked)
    4th Trial - Options1(Remove only Damaging) + Naive Positive + Naive Damaging (checked)
    
    5th Trial - Options2(Remove damaging + irrelevant) + Strict Positive + Strict Damaging (checked)
        - The one also represents other three cases
            - Options2(Remove damaging + irrelevant) + Strict Positive + Naive Damaging 
            - Options2(Remove damaging + irrelevant) + Naive Positive + Strict Damaging (checked)
            - Options2(Remove damaging + irrelevant) + Naive Positive + Naive Damaging 
    
    6th Trial - Options3(Removes damaging + relevant) + Strict Positive + Strict Damaging (checked)
    7th Trial - Options3(Removes damaging + relevant) + Strict Positive + Naive Damaging (checked)
    8th Trial - Options3(Removes damaging + relevant) + Naive Positive + Strict Damaging (checked)
    9th Trial - Options3(Removes damaging + relevant) + Naive Positive + Naive Damaging (checked)
    
    10th Trial - Options4(Removes damaging + irrelevant + relevant) + Strict Positive (checked)
        - The one represents cases 
            - Options4(Removes damaging + irrelevant + relevant) + Strict Positive + Strict Damaging (checked)
            - Options4(Removes damaging + irrelevant + relevant) + Strict Positive + Naive Damaging
        - almost equal to "pos1_ctx1" e.g.) 01011 -> 2 strict positive cases
        - "pos1_ctx1" = baseline could be our baselines
        
    11th Trial - Options4(Removes damaging + irrelevant + relevant) + Naive Positive (checked)  
        - The one represents cases 
            - Options4(Removes damaging + irrelevant + relevant) + Naive Positive + Strict Damaging
            - Options4(Removes damaging + irrelevant + relevant) + Naive Positive + Naive Damaging (checked)

# Implementation

In [4]:
def build_data(input_file, option, option_p, option_d, sample_size):
    '''
    input_file : incremental inference result from FiD from KILT-5-1
        path : /data/philhoon-relevance/FiD/results/KILT_BM25_NQ/incremental_result/pos1_ctx5.json
        
    output : FiD input json format
    
    option(required) : removing strategies
        op1 : removes damages only
        op2 : removes damaging + irrelevant
        op3 : removes damaging + relevant
        op4 : removes damaging + irrelevant + relevant
        
    option_p(required) : positive passage selection options
        strict : strict positive
            e.g.) 11 pattern 
                1st '1' is positive, 2nd '1' is relevant
        naive : naive positive
            e.g.) 11 pattern 
                1st '1' is positive, 2nd '1' is positive
                
    option_d(required) : damaging passage selection options
        strict : strict negative
            e.g.) A00 pattern 
                if there is at least one '1' occurred in A, 2nd '0' is irrelevant
        naive : naive damaging
            e.g.) A00 pattern 
                if there is at least one '1' occurred in A, 2nd '0' is damaging
    
    '''
    
    output_format = []
    null_em = '0' * sample_size
    
    # 'strict', 'naive'  
    # option_p = 'naive'
    # option_d = 'naive'
    # option = 'op4'

    for id_, instance in enumerate(input_file,1):
        template_dict = {}
        if 'id' in instance.keys():
            template_dict['id'] = instance['id']
        else:
            template_dict['id'] = str(id_)
        template_dict['answers'] = instance['answers']
        template_dict['question'] = instance['question']
        template_dict['em_pattern'] = instance['em_pattern']
        template_dict['sentence_inference'] = instance['sentence_inference']

        em_pattern = instance['sentence_inference']

        # when there is at least one EM in the accumulated inference
        if em_pattern != null_em:   
            new_ctx = []

            # relevant vs positive
            positve_ctx_lst = []
            relevant_ctx_lst = []

            # irrelevant vs damaging
            damaging_ctx_lst = []
            irrelevant_ctx_lst = []


            for idx_, ctx in enumerate(instance['ctxs']):

                # checking current em
                cur_em = em_pattern[idx_]
                pre_em_pattern = em_pattern[:idx_]


                # first 1 : positive
                if not pre_em_pattern and cur_em == '1':
                    positve_ctx_lst.append(ctx)

                # first 0 : irrelevant
                elif not pre_em_pattern and cur_em == '0':
                    irrelevant_ctx_lst.append(ctx)
                    
                # 01 pattern : positive 
                elif pre_em_pattern and pre_em_pattern[-1] == '0' and cur_em == '1':
                    positve_ctx_lst.append(ctx)

                # 10 pattern : damaging
                elif pre_em_pattern and pre_em_pattern[-1] == '1' and cur_em == '0':
                    damaging_ctx_lst.append(ctx)

                # 11 pattern : Strict Positive(relevant) or Naive Positive(positive)
                elif pre_em_pattern and pre_em_pattern[-1] == '1' and cur_em == '1':
                    if option_p == 'strict':
                        relevant_ctx_lst.append(ctx)

                    elif option_p == 'naive':
                        positve_ctx_lst.append(ctx)

                    else:
                        print('option_p should be either \'strict\' or \'naive\'')
                        return 

                # 00 pattern : Strict Damaging(irrelevant) or Naive Damaging(damaging) 
                elif pre_em_pattern and pre_em_pattern[-1] == '0' and cur_em == '0':
                    # if '1' does not occured in A, currnet passage is irrelevant
                    if not '1' in pre_em_pattern:
                        irrelevant_ctx_lst.append(ctx)

                    # if '1' occurred in A, 
                    else:
                        # strict : consider it as irrelevnat 
                        if option_d == 'strict':
                            irrelevant_ctx_lst.append(ctx)

                        # naive : consider it as damaging 
                        elif option_d == 'naive':
                            damaging_ctx_lst.append(ctx)

                        else:
                            print('option_p should be either \'strict\' or \'naive\'')
                            return 

            # op1 removes damages only
            if option == 'op1':
                new_ctx.extend(positve_ctx_lst)
                new_ctx.extend(relevant_ctx_lst)
                new_ctx.extend(irrelevant_ctx_lst)


            # op2 removes damaging + irrelevant
            elif option == 'op2':
                new_ctx.extend(positve_ctx_lst)
                new_ctx.extend(relevant_ctx_lst)

            # op3 : Removes damaging + relevant
            elif option == 'op3':
                new_ctx.extend(positve_ctx_lst)
                new_ctx.extend(irrelevant_ctx_lst)

            # op4 : Removes damaging + irrelevant + relevant
            elif option == 'op4':
                new_ctx.extend(positve_ctx_lst)

            else:
                print('option should be op1, op2, op3, op4')
                return 

            template_dict['ctxs'] = new_ctx
            output_format.append(template_dict)

        # when there is no EM in the accumulated inference
        else:
            template_dict['ctxs']= instance['ctxs']
            output_format.append(template_dict)
    
    print('==============instance finished======================')
    return output_format
        


In [5]:
# option_dict = {
#     'op1' : 'remove_damage',
#     'op2' : 'remove_damage_irrelevant',
#     'op3' : 'remove_damage_relevant',
#     'op4' : 'remove_damage_irrelevant_relevant',
# }
# option = 'op4'

In [6]:
# op1, op2 are duplicates
method_option_dict = {
    'op3' : 'remove_damage_relevant',
    'op4' : 'remove_damage_irrelevant_relevant',
}

In [7]:
option_p_dict = {
    'strict' : 'strict_positive',
    'naive' : 'naive_positive',
}
# option_p = 'strict'

In [8]:
option_d_dict = {
    'strict' : 'strict_damaging',
    'naive' : 'naive_damaging',
}
# option_d = 'strict'

In [9]:
# input_file = utils.open_json('/data/philhoon-relevance/FiD/results/KILT_BM25_NQ/incremental_result/pos1_ctx5.json')
# output_path = '/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_BM25_SELECTION'

## NQ TEST DPR 

In [10]:
# sample_size = 100
# input_file = f'/data/philhoon-relevance/FiD/results/NQ_DPR/TEST/incremental_result_{sample_size}/ctx{sample_size}.json'
# output_path = f'/data/philhoon-relevance/FiD/open_domain_data/NQ_TEST_DPR_SELECTION/'

## NQ DEV Contriever

In [11]:
# sample_size = 100
# # /data/philhoon-relevance/FiD/results/NQ_CONTRIEVER/DEV/incremental_result_100
# input_file = f'/data/philhoon-relevance/FiD/results/NQ_CONTRIEVER/DEV/incremental_result_{sample_size}/ctx{sample_size}.json'
# output_path = f'/data/philhoon-relevance/FiD/open_domain_data/NQ_DEV_CONTRIEVER_SELECTION/'

## FiD-encoder-sentence-level-classifier Prediction

In [12]:
sample_size = 100
# /data/philhoon-relevance/FiD/results/NQ_CONTRIEVER/DEV/incremental_result_100
input_file = f'/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/FiD-Encoder-lstm-12layers-sequence_exclude_no_answer_partial_decisive/step_2160/test_prediction/ctx100id_test_prediction.json'
output_path = f'/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/FiD-Encoder-lstm-12layers-sequence_exclude_no_answer_partial_decisive/step_2160/test_prediction/PREDICTION_SELECTION'





In [13]:
# NQ_DEV_CONTRIEVER_SELECTION

In [14]:
input_ = utils.open_json(input_file)

In [15]:
output_path

'/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/FiD-Encoder-lstm-12layers-sequence_exclude_no_answer_partial_decisive/step_2160/test_prediction/PREDICTION_SELECTION'

In [16]:
len(input_)

3610

In [17]:
input_[0]['sentence_inference']

'1000100100100100100001001001001001001000000000010010000000001001001000000010010000000010010010010001'

In [19]:
for o_ in method_option_dict.keys():
    for op in option_p_dict.keys():
        for od in option_d_dict.keys():
            option = o_
            option_p = op
            option_d = od
            
            if option == 'op4' and option_p == 'strict' and option_d == 'strict':
                continue
            if option == 'op4' and option_p == 'naive' and option_d == 'strict':
                continue 
            
            filename = f'{option_p_dict[option_p]}_{option_d_dict[option_d]}_{method_option_dict[option]}.json'

            output_file = os.path.join(output_path, filename)
            output_format = build_data(input_, option, option_p, option_d, sample_size)
            
            utils.save_json(output_format, output_file)
            print(f'{filename} save on \n {output_path}')
#             print(f'option : {option}')
#             print(f'option_p : {option_p}')
#             print(f'option_d : {option_d}')
#             print(f'filename : {filename}')
#             print(f'output_file : {output_file}')
            

==============instance finished======================
strict_positive_strict_damaging_remove_damage_relevant.json save on 
 /data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/FiD-Encoder-lstm-12layers-sequence_exclude_no_answer_partial_decisive/step_2160/test_prediction/PREDICTION_SELECTION
==============instance finished======================
strict_positive_naive_damaging_remove_damage_relevant.json save on 
 /data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/FiD-Encoder-lstm-12layers-sequence_exclude_no_answer_partial_decisive/step_2160/test_prediction/PREDICTION_SELECTION
==============instance finished======================
naive_positive_strict_damaging_remove_damage_relevant.json save on 
 /data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/FiD-Encoder-lstm-12layers-sequence_exclude_no_answer_partial_decisive/step_2160/test_prediction/PREDICTION_SELECTION
==============instance finished======================


In [23]:
test_path= '/data/philhoon-relevance/binary-classification/results/NQ-DEV-DPR/5-fold/1/FiD-Encoder-lstm-12layers-sequence_exclude_no_answer_partial_decisive/step_2160/test_prediction/PREDICTION_SELECTION'
# method 2
test_file = 'naive_positive_naive_damaging_remove_damage_irrelevant_relevant.json'
test_file_path = os.path.join(test_path, test_file)

In [24]:
test_data = utils.open_json(test_file_path)
print(test_data[0].keys())
print(len(test_data))

dict_keys(['id', 'answers', 'question', 'em_pattern', 'sentence_inference', 'ctxs'])
3610


In [29]:
for instance in test_data:
    print(instance['em_pattern'])
    print(instance['sentence_inference'])
    print(instance['sentence_inference'].count('1'))
    print(len(instance['ctxs']))
    print('----')

0000111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
1000100100100100100001001001001001001000000000010010000000001001001000000010010000000010010010010001
24
24
----
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
0101001001001001001001001001001000010010010010001000000101100000000010010010000010100100100000000010
27
27
----
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
1001001001000101001001001000101001000001000000001001001001001001001000010010100100001000010000000100
27
27
----
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
1000000000010010010010010010010000000000000000000000001000000001001001000000010010000000000000000000
14
14
----
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
00000000100100010000000000100110010010010000000

# Method explanation
- Redundancies in selection strategies -> remove them -> 6

#### Method1. Include passages that corresponds to 1s
- all __option2__ belong to this case
- option4_naive_naive, option4_naive_strict

'00011011000'  -> 000<span style="color:red">1</span><span style="color:red">1</span>0<span style="color:red">1</span><span style="color:red">1</span>000  
- red 1s

#### Method2. Include passages that corresponds to first 1s
- option4_strict_naive, option4_strict_strcit 

'00011011000'  -> 
000<span style="color:red">1</span>10<span style="color:red">1</span>1000  
- red 1s
- removing consecutive passages when the previous output is correct as well

#### Method3. Include passages that corresponds to 1s + Included First appeared consecutive 0s
- option1_naive_naive, option1_strict_naive
- option3_naive_naive

'00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span></span><span style="color:red">1</span>0<span style="color:red">1</span></span><span style="color:red">1</span>000  
- red 1s and blue 0s
- usually top-retrieved results contain the answer 

#### Method4. Include passages that corresponds to first 1s + Included First appeared consecutive 0s
- option3_strict_naive

'00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span>10<span style="color:red">1</span>1000  
- red 1s and blue 0s
- removing consecutive passages when the previous output is correct as well
- usually top-retrieved results contain the answer 

#### Method5. Include passages that corresponds to 1s + Remove only damaging
- option1_naive_strict, option1_strict_strict
- option3_naive_strict

'00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span></span><span style="color:red">1</span>0<span style="color:red">1</span></span><span style="color:red">1</span>0<span style="color:blue">0</span><span style="color:blue"><span style="color:blue">0</span><span style="color:blue"> 
- red 1s and blue 0s
- only removes damaging passages for comparison with Method3, Method4
- usually top-retrieved results contain the answer 

#### Method6. Include passages that corresponds to first 1s  + Remove only damaging
- option3_strict_strict

'00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span>10<span style="color:red">1</span>10<span style="color:blue">0</span><span style="color:blue">0</span>
- red 1s and blue 0s
- removing consecutive passages when the previous output is correct as well
- only removes damaging passages for comparison with Method3, Method4
- usually top-retrieved results contain the answer 

## Conclusion
- option1, option2 are not needed
- Method1 : option4_naive_naive
- Method2 : option4_strict_naive 
- Method3 : option3_naive_naive
- Method4 : option3_strict_naive
- Method5 : option3_naive_strict
- Method6 : option3_strict_strict